In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys 

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

import cvxpy as cp 
import numpy as np 
import pandas as pd 

from src.setup_problem_dual import Setup
from src.setup_problem_dual import ObjectiveFunction
from src.setup_problem_dual import Predicate_dual

In [3]:
data_dir_path = "./../../inputs/toy_data_loss" # たまたまファイル名が都合よいだけで，loss 関数は関係がない

file_names_dict = {
    'supervised': ['L_p1.csv', 'L_p2.csv', 'L_p3.csv'],
    'unsupervised': ['U.csv'],
    'rule': ['rules.txt']
}

problem_instance = Setup(data_dir_path, file_names_dict, ObjectiveFunction)
objective, constraints = problem_instance.main()

problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)

load_data took 0.008779764175415039 seconds!
load_rules took 0.027861595153808594 seconds!
formulate_predicates_with_cvxpy took 0.000415802001953125 seconds!
construct_constraints took 0.03774714469909668 seconds!
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 23 11:46:16 AM: Your problem has 96 variables, 135 constraints, and 0 parameters.
(CVXPY) Dec 23 11:46:16 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 23 11:46:16 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 23 11:46:16 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
----------------------------

# 切片 $b_j$ の値の調整ができていない気がする 

In [8]:
p1 = Predicate_dual(problem_instance, 'p1')
X = problem_instance.L['p1'][:, :-1]
Y = problem_instance.L['p1'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p1(x))

I am a predicate (dual)
[0.1 0.5] -1.0 -0.6749766844491083
[0.4 0.4] -1.0 -0.6499872053229678
[0.3 0.8] 1.0 0.07499165368733496
[0.9 0.7] 1.0 0.3249638897720766


In [5]:
p2 = Predicate_dual(problem_instance, 'p2')
X = problem_instance.L['p2'][:, :-1]
Y = problem_instance.L['p2'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p2(x))

I am a predicate (dual)
[0.1 0.3] -1.0 -0.17769613882067903
[0.6 0.4] -1.0 -0.0676921794352277
[0.2 0.8] 1.0 0.1692319641528714
[0.7 0.6] 1.0 0.07615635410303517


In [6]:
p3 = Predicate_dual(problem_instance, 'p3')
X = problem_instance.L['p3'][:, :-1]
Y = problem_instance.L['p3'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p3(x))

I am a predicate (dual)
[0.4 0.2] -1.0 -0.12499508088888082
[0.9 0.3] -1.0 -0.07502502963705804
[0.2 0.6] 1.0 0.07501894678842355
[0.5 0.7] 1.0 0.1250011637375155


# $b_j$ に 0.5 を足してみる (0 を閾値にする)

In [18]:
p1 = Predicate_dual(problem_instance, 'p1')
X = problem_instance.L['p1'][:, :-1]
Y = problem_instance.L['p1'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p1(x))

b: -1.7499373259309368
[0.1 0.5] -1.0 -0.1749766844491083
[0.4 0.4] -1.0 -0.14998720532296783
[0.3 0.8] 1.0 0.574991653687335
[0.9 0.7] 1.0 0.8249638897720766


In [19]:
p2 = Predicate_dual(problem_instance, 'p2')
X = problem_instance.L['p2'][:, :-1]
Y = problem_instance.L['p2'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p2(x))

b: -0.38923786217070516
[0.1 0.3] -1.0 0.32230386117932097
[0.6 0.4] -1.0 0.4323078205647723
[0.2 0.8] 1.0 0.6692319641528715
[0.7 0.6] 1.0 0.5761563541030352


In [20]:
p3 = Predicate_dual(problem_instance, 'p3')
X = problem_instance.L['p3'][:, :-1]
Y = problem_instance.L['p3'][:, -1]
for x, y in zip(X, Y):
    print(x, y, p3(x))

b: -0.22497168048434366
[0.4 0.2] -1.0 0.37500491911111916
[0.9 0.3] -1.0 0.42497497036294196
[0.2 0.6] 1.0 0.5750189467884236
[0.5 0.7] 1.0 0.6250011637375155


# カーネルを非線形のものにして，pima indian に対して適用してみる

In [21]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4.txt']
}

In [23]:
problem_instance = Setup(data_dir_path, file_names_dict, ObjectiveFunction)
objective, constraints = problem_instance.main()

problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/L_Pregnancies_High.csv'